### A basic program to understand Tensorflow Basics

In [16]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy as np

In [20]:
### Import dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data",one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### PlaceHolder
- Placeholders in Tensorflow basically specifies those variables whose values you will specify later when you run the Tensorflow session
- It will be typically used to specify the input and the output data

In [36]:
## Hyperparameters
learning_rate = 0.01
num_steps = 1000
batch_size = 1024
display_step = 100

## Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

## Input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

### Specifying variable in tensorflow
- Using ```tf.Variable```
- h1,h2,out are the weight matrices for layer 1,2,output respectively
- b1,b2,out are the biase matrices for the layer 1,2,output respectively

In [37]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

### Specifying the NN Model
- It is actually just ultimately creating the cost function
    - The out layer function is just a f(x) , ie. the cost function
    - Where x in the input

In [38]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

### Loss and Optimizer 
- logits?
    - It is a the vector of raw (non-normalized) predictions that a classification model generates
- loss_op
    - This operation calculated the loss of the neural network on the given input
    - This the single number evaluation metric that tells us how good our NN is doing
    - And this is the function that we have to optimize ( reduce to get a good NN )!
- optimizer
    - This is the optimizer that will optimize our loss function
    - We can use RMS Prop , Grad Descent , Adam 
- train_op
    - Optimizer is trying to optimize the loss_op (ie the loss function)
   

In [39]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

### Run Session
- ```sess.run()``` feed your mini-batch using the previously defined place-holder 
- Pass the batches data while runnning the session

In [40]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

Step 1, Minibatch Loss= 2460.1982, Training Accuracy= 0.198
Step 100, Minibatch Loss= 85.7056, Training Accuracy= 0.896
Step 200, Minibatch Loss= 48.8837, Training Accuracy= 0.895
Step 300, Minibatch Loss= 28.7514, Training Accuracy= 0.908
Step 400, Minibatch Loss= 21.6062, Training Accuracy= 0.901
Step 500, Minibatch Loss= 25.2531, Training Accuracy= 0.898
Step 600, Minibatch Loss= 23.7964, Training Accuracy= 0.890
Step 700, Minibatch Loss= 16.4889, Training Accuracy= 0.900
Step 800, Minibatch Loss= 16.7202, Training Accuracy= 0.910
Step 900, Minibatch Loss= 17.8916, Training Accuracy= 0.905
Step 1000, Minibatch Loss= 14.0190, Training Accuracy= 0.911
Optimization Finished!
Testing Accuracy: 0.8825
